In [ ]:
import numpy as np
from numba import jit
from special import *

In [ ]:
import matplotlib.pyplot as plt
%matplotlib

In [ ]:
def f3ver1(j,e1,e2,e3,N):
    return ((np.exp(N*e3)/(e3**j) - np.exp(N*e2)/(e2**j))*e1+\
         (np.exp(N*e1)/(e1**j) - np.exp(N*e3)/(e3**j))*e2+\
         (np.exp(N*e2)/(e2**j) - np.exp(N*e1)/(e1**j))*e3)/\
        ((e1-e2)*(e2-e3)*(e3-e1))    

In [ ]:
def f3ver2(j,eps1,eps2,eps3,N):
    e1=eps1
    e2=eps2
    e3=eps3
    #if abs(e1)<1.0:
    if abs(e2) >= abs(e1) and abs(e2)>= abs(e3):
        e1, e2, e3 = e2, e1, e3
    if abs(e3) >= abs(e1) and abs(e3) >= abs(e2):
        e1, e2, e3 = e3, e2, e1
    print('e1',e1,'e2',e2,'e3',e3)
    kmax=61
    tol=10**-12
    if j==1:
        out=0.0
        for k in range(0,kmax):
            prod=np.exp(N*e1)*(-e1)**(-k-3)
            summ=0.0
            for m in range(0,k+2+1):
                summ=summ+(-N*e1)**m/fact[m]
            prod=prod*summ
            summ=0.0
            for m in range(0,k+1):
                summ=summ+(e2-e1)**(k-m) * (e3-e1)**m
            temp=prod*summ
            out=out + temp
            #print('k',k,'val',temp)
            if k>5 and abs(temp) < abs(out)*tol:
                break
    elif j==2:
        out=0.0
        for k in range(0,kmax):
            prod=-np.exp(N*e1)*(-e1)**(-k-4)
            summ=0.0
            for m in range(0,k+2+1):
                summ=summ+(-N*e1)**m *(k-m+3)/fact[m]
            prod=prod*summ
            summ=0.0
            for m in range(0,k+1):
                summ=summ+(e2-e1)**(k-m) * (e3-e1)**m
            temp=prod*summ
            out=out + temp
            if k>5 and abs(temp) < abs(out)*tol:
                break
    print('k',k)
    return out

In [ ]:
def f3ver3(j,eps1,eps2,eps3,N):
    e1,e2,e3 = arrange(eps1,eps2,eps3)
    return ((np.exp(N*e3)/(e3**j)) + e2*e1*f2(j+1,e1,e2,N)-e3*f2(j,e1,e2,N))/\
            ((e2-e3)*(e3-e1))

In [ ]:
def f3ver4(j,eps1,eps2,eps3,N):
    e1=eps1
    e2=eps2
    e3=eps3
    if abs(e2) <= abs(e1) and abs(e2) <= abs(e3):
        e1, e2, e3 = e2, e1, e3
    if abs(e3) <= abs(e1) and abs(e3) <= abs(e2):
        e1, e2, e3 = e3, e2, e1    
    return -1.0/(e2*e3*(e1**j))

In [ ]:
npts=100
out1=[]
out2=[]
out3=[]
out4=[]
N=10.32
j=2
delta=np.logspace(-8,0,npts)
for ii in range(0,npts):
    e1=-2*delta[ii]
    e2=e1-delta[ii]
    e3=e2-4*delta[ii]
    out1.append(f3ver1(j,e1,e2,e3,N))
    out2.append(f3ver2(j,e1,e2,e3,N))
    out3.append(f3ver3(j,e1,e2,e3,N))
    out4.append(f3ver4(j,e1,e2,e3,N))

In [ ]:
y=abs(np.array(out1)-np.array(out2))
plt.loglog(delta,y)
plt.show()

In [ ]:
y=abs(np.array(out1))

plt.loglog(delta,y)
plt.show()

# Series Convergance

In [ ]:
e1=-0.0003
delta=0.0003
e2=e1-delta
e3=e2-20*delta
terms=[]
ks=[]

if abs(e1)<1.0:
    if abs(e2) >= abs(e1) and abs(e2)>= abs(e3):
        e1, e2, e3 = e2, e1, e3
    elif abs(e3) >= abs(e1) and abs(e3) >= abs(e2):
        e1, e2, e3 = e3, e2, e1
    else:
        print('same')
        
for k in range(0,80):
    ks.append(k)
    prod=-np.exp(N*e1)*(-e1)**(-k-4)
    summ=0.0
    for m in range(0,k+2+1):
        summ=summ+(-N*e1)**m *(k-m+3)/fact[m]
    prod=prod*summ
    summ=0.0
    for m in range(0,k+1):
        summ=summ+(e2-e1)**(k-m) * (e3-e1)**m
    terms.append(abs(prod*summ))

In [ ]:
plt.semilogy(ks,terms,'.')
plt.show()

# Test f2

In [ ]:
npts=1000
N=10
e2=0.001
j=2
e1s=np.logspace(-7,1,npts)
out=[]
for ii in range(0,npts):
    out.append(abs(f2(j,e1s[ii],2*e1s[ii],N)))
    

In [ ]:
plt.loglog(e1s,out)
plt.show()

# Between rock and hard place

In [ ]:
def f3nearZero(j,e1,e2,e3,N):
    return ((np.exp(N*e3)/(e3**j)) + e2*e1*f2(j+1,e1,e2,N) - e3*f2(j,e1,e2,N))/\
            ((e2-e3)*(e3-e1))

In [ ]:
e1=-0.23
e2=-0.10
npts=200
N=10
j=2
delta=np.logspace(-10,-3,npts)
f2based=[]
series=[]
origional=[]
for ii in range(0,npts):
    e1=-0.1
    e2=-0.001
    e3=e2-delta[ii]
    f2based.append(abs(f3nearZero(j,e1,e2,e3,N)))
    series.append(abs(f3ver2(j,e1,e2,e3,N)))
    origional.append(abs(f3ver1(j,e1,e2,e3,N)))
    
f2based=np.array(f2based)
series=np.array(series)
origional=np.array(origional)

In [ ]:
plt.semilogy(abs(delta),f2based,label='f2based')
plt.semilogy(abs(delta),series,label='series')
plt.semilogy(abs(delta),origional,'--',label='origional')
plt.legend()
plt.show()

In [ ]:
plt.semilogy(abs(delta),abs(f2based-series),label='f2based')
plt.show()

In [ ]:
e1=-0.10001
e2=-0.010002
e3=-0.02
N=0.0001
j=1
print('f2based  ',f3nearZero(j,e1,e3,e2,N))
print('series   ',f3ver2(j,e1,e2,e3,N))
print('origional',f3ver1(j,e1,e2,e3,N))
print('f3',f3(j,e1,e2,e3,N))

In [ ]:
e1=-0.01
e2=-10.1
e3=-10.2
N=10
j=1
print('f2based  ',f3nearZero(j,e1,e2,e3,N))
print('series   ',f3ver2(j,e1,e2,e3,N))
print('origional',f3ver1(j,e1,e2,e3,N))
print('f3',f3(j,e1,e2,e3,N))